## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Conde,-7.2597,-34.9075,78.26,94,75,3.44,broken clouds,BR,2022-03-27 22:13:33
1,1,Atuona,-9.8000,-139.0333,79.07,77,52,14.43,broken clouds,PF,2022-03-27 22:13:33
2,2,Bluff,-46.6000,168.3333,56.16,82,11,3.98,few clouds,NZ,2022-03-27 22:13:33
3,3,Nikolskoye,59.7035,30.7861,26.71,80,100,7.81,overcast clouds,RU,2022-03-27 22:09:36
4,4,Kodiak,57.7900,-152.4072,33.31,100,100,31.07,heavy snow,US,2022-03-27 22:09:57


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Conde,-7.2597,-34.9075,78.26,94,75,3.44,broken clouds,BR,2022-03-27 22:13:33
1,1,Atuona,-9.8000,-139.0333,79.07,77,52,14.43,broken clouds,PF,2022-03-27 22:13:33
7,7,Iquitos,-3.7481,-73.2472,75.24,94,75,3.44,broken clouds,PE,2022-03-27 22:13:35
9,9,Puerto Ayora,-0.7393,-90.3518,84.16,76,96,12.24,light rain,EC,2022-03-27 22:13:35
12,12,Road Town,18.4167,-64.6167,78.89,73,40,5.75,scattered clouds,VG,2022-03-27 22:13:36


In [9]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                210
City                   210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
Country                210
Date                   210
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Conde,BR,78.26,broken clouds,-7.2597,-34.9075,
1,Atuona,PF,79.07,broken clouds,-9.8000,-139.0333,
7,Iquitos,PE,75.24,broken clouds,-3.7481,-73.2472,
9,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,
12,Road Town,VG,78.89,scattered clouds,18.4167,-64.6167,
18,Bajil,YE,78.75,broken clouds,15.0584,43.2851,
31,Ca Mau,VN,78.82,scattered clouds,9.1769,105.1500,
32,Voh,NC,81.30,scattered clouds,-20.9667,164.7000,
38,Santiago De Las Vegas,CU,76.71,clear sky,22.9700,-82.3869,
42,Rikitea,PF,79.11,clear sky,-23.1203,-134.9692,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))